In [38]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


import os
for dirname, _, filenames in os.walk('C://Users/ismail/Documents/TP5/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


C://Users/ismail/Documents/TP5/calendar.csv
C://Users/ismail/Documents/TP5/sales_train_evaluation.csv
C://Users/ismail/Documents/TP5/sales_train_validation.csv
C://Users/ismail/Documents/TP5/sample_submission.csv
C://Users/ismail/Documents/TP5/sell_prices.csv


In [39]:
df_cal = pd.read_csv('C://Users/ismail/Documents/TP5/calendar.csv')
df_eval = pd.read_csv('C://Users/ismail/Documents/TP5/sales_train_evaluation.csv')
df_price = pd.read_csv('C://Users/ismail/Documents/TP5/sell_prices.csv')
df_sample_output = pd.read_csv('C://Users/ismail/Documents/TP5/sample_submission.csv')

In [40]:
holiday = ['NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay', 'SuperBowl', 'PresidentsDay', 'StPatricksDay', 'Easter', 'Cinco De Mayo', 'IndependenceDay', 'EidAlAdha', 'Thanksgiving', 'Christmas']
weekend = ['Saturday', 'Sunday']

df_cal['is_holiday_1'] = df_cal['event_name_1'].apply(lambda x : 1 if x in holiday else 0 )
df_cal['is_holiday_2'] = df_cal['event_name_1'].apply(lambda x : 1 if x in holiday else 0 )
df_cal['is_holiday'] = df_cal[['is_holiday_1','is_holiday_2']].max(axis=1)
df_cal['is_weekend'] = df_cal['weekday'].apply(lambda x : 1 if x in weekend else 0 )

In [41]:
df_cal = df_cal.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')

In [42]:
del_col = []
for x in range(1851):
    del_col.append('d_' + str(x+1))

In [43]:
df_eval = df_eval.drop(del_col, axis='columns')

In [44]:
df_eval = df_eval.melt(['id','item_id','dept_id','cat_id','store_id','state_id'], var_name='d', value_name='qty')
print(df_eval.shape)
df_eval.head()

(2744100, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,qty
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,1
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1852,0


In [45]:
df_eval = pd.merge(df_eval, df_cal, how='left', on='d')

In [46]:
df_eval = pd.merge(df_eval, df_price, how='left', on=['item_id', 'wm_yr_wk', 'store_id'])


In [47]:
df_eval_test = df_eval.query('d == "d_1852"')

In [48]:
df_eval_test = df_eval_test[['id', 'store_id', 'item_id', 'dept_id', 'cat_id', 'state_id', 'd', 'qty', 'sell_price']]

In [49]:
df_eval_test['qty'] = df_eval_test['d'].apply(lambda x: int(x.replace(x, '0')))

In [50]:
tmp_df = df_eval_test

In [51]:
for x in range(28):
    df_eval_test = df_eval_test.append(tmp_df)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_9312\507363638.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eval_test = df_eval_test.append(tmp_df)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_9312\507363638.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eval_test = df_eval_test.append(tmp_df)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_9312\507363638.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eval_test = df_eval_test.append(tmp_df)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_9312\507363638.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eval

In [52]:
df_eval_test = df_eval_test.reset_index(drop=True)

In [53]:
lst_d = []
i = 0
lst_index = df_eval_test.index
for x in lst_index:
    lst_d.append('d_' + str(((lst_index[i]) // 30490) + 1942))
    i = i + 1

In [54]:
df_eval_test['d'] = lst_d

In [55]:
df_eval_test = pd.merge(df_eval_test, df_cal, how='left', on='d')

In [56]:
df_eval_test = pd.merge(df_eval_test, df_price, how='left', on=['item_id', 'wm_yr_wk', 'store_id'])

In [57]:
import gc
del tmp_df
gc.collect()

202

In [58]:
df_eval = pd.get_dummies(data=df_eval, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])
df_eval_test = pd.get_dummies(data=df_eval_test, columns=['dept_id', 'cat_id', 'store_id', 'state_id'])

In [59]:
df_eval_test = df_eval_test.drop(['sell_price_x', 'snap_CA', 'snap_TX', 'snap_WI'], axis='columns')
df_eval_test = df_eval_test.rename(columns={'sell_price_y': 'sell_price'})
df_eval = df_eval.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis='columns')                                   

In [60]:
from sklearn.model_selection import train_test_split
target_col = 'qty'
exclude_cols = ['id', 'item_id', 'd', 'date', 'wm_yr_wk']
feature_cols = [col for col in df_eval.columns if col not in exclude_cols]
y = np.array(df_eval[target_col])
X = np.array(df_eval[feature_cols])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [62]:
pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [63]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)


params = {
        'n_jobs': -1,
        'boosting_type': 'gbdt',
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 64,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : 1,
        "seed": 42,
        }


evals_result = {}

model = lgb.train(params, lgb_train, 5000, 
                      valid_sets=[lgb_eval], 
                      early_stopping_rounds=1500, 
                      verbose_eval=50, 
                      evals_result=evals_result)

C:\Users\ismail\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ismail\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ismail\anaconda3\lib\site-packages\lightgbm\engine.py:260: UserWarning: 'evals_result' argument is deprecated and will be removed in a future release of LightGBM. Pass 'record_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'evals_result' argument is d

[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.414929 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 379
[LightGBM] [Info] Number of data points in the train set: 2058075, number of used features: 29
[LightGBM] [Warning] Unknown parameter: bagging_frequency
[LightGBM] [Info] Start training from score 1.395218
Training until validation scores don't improve for 1500 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	valid_0's rmse: 3.1763
[100]	valid_0's rmse: 2.75736
[150]	valid_0's rmse: 2.4468
[200]	valid_0's rmse: 2.17382
[250]	valid_0's rmse: 1.91874
[300]	valid_0's rmse: 1.71201
[350]	valid_0's rmse: 1.50441
[400]	valid_0's rmse: 1.36319
[450]	valid_0's rmse: 1.22157
[500]	valid

In [64]:
pred = model.predict(df_eval_test[feature_cols])

In [65]:
df_eval_test['pred_qty'] = pred

In [66]:
predictions = df_eval_test[['id', 'date', 'pred_qty']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'pred_qty').reset_index()
predictions

date,id,NaN,2016-05-23,2016-05-24,2016-05-25,2016-05-26,2016-05-27,2016-05-28,2016-05-29,2016-05-30,...,2016-06-10,2016-06-11,2016-06-12,2016-06-13,2016-06-14,2016-06-15,2016-06-16,2016-06-17,2016-06-18,2016-06-19
0,FOODS_1_001_CA_1_evaluation,0.084437,0.000615,0.000615,0.000615,0.000615,0.000615,-0.006474,-0.006474,0.000615,...,0.000615,-0.006474,-0.006474,0.000615,0.000615,0.000615,0.000615,0.000615,-0.006474,-0.006474
1,FOODS_1_001_CA_2_evaluation,0.035619,0.018037,0.018037,0.018037,0.018037,0.018037,0.019361,0.019361,0.018037,...,0.018037,0.019361,0.019361,0.018037,0.018037,0.018037,0.018037,0.018037,0.019361,0.019361
2,FOODS_1_001_CA_3_evaluation,0.348475,0.002899,0.002899,0.002899,0.002899,0.002899,0.001234,0.001234,0.002899,...,0.002899,0.001234,0.001234,0.002899,0.002899,0.002899,0.002899,0.002899,0.001234,0.001234
3,FOODS_1_001_CA_4_evaluation,0.004666,0.003304,0.003304,0.003304,0.003304,0.003304,-0.003908,-0.003908,0.003304,...,0.003304,-0.003908,-0.003908,0.003304,0.003304,0.003304,0.003304,0.003304,-0.003908,-0.003908
4,FOODS_1_001_TX_1_evaluation,0.025422,-0.004310,-0.004310,-0.004310,-0.004310,-0.004310,-0.005741,-0.005741,-0.004310,...,-0.004310,-0.005741,-0.005741,-0.004310,-0.004310,-0.004310,-0.004310,-0.004310,-0.005741,-0.005741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,-0.125231,-0.002891,-0.002891,-0.002891,-0.002891,-0.002891,-0.003881,-0.003881,-0.002891,...,-0.002891,-0.003881,-0.003881,-0.002891,-0.002891,-0.002891,-0.002891,-0.002891,-0.003881,-0.003881
30486,HOUSEHOLD_2_516_TX_3_evaluation,-0.088636,-0.003180,-0.003180,-0.003180,-0.003180,-0.003180,-0.005773,-0.005773,-0.003180,...,-0.003180,-0.005773,-0.005773,-0.003180,-0.003180,-0.003180,-0.003180,-0.003180,-0.005773,-0.005773
30487,HOUSEHOLD_2_516_WI_1_evaluation,-0.120247,-0.001506,-0.001506,-0.001506,-0.001506,-0.001506,-0.003348,-0.003348,-0.001506,...,-0.001506,-0.003348,-0.003348,-0.001506,-0.001506,-0.001506,-0.001506,-0.001506,-0.003348,-0.003348
30488,HOUSEHOLD_2_516_WI_2_evaluation,0.931745,-0.002552,-0.002552,-0.002552,-0.002552,-0.002552,-0.004662,-0.004662,-0.002552,...,-0.002552,-0.004662,-0.004662,-0.002552,-0.002552,-0.002552,-0.002552,-0.002552,-0.004662,-0.004662


In [67]:
predictions = predictions.drop(predictions.columns[1], axis=1)

In [68]:
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]


In [69]:
x = 2744099 + 1 - 853720
df_val = df_eval[x:]

In [70]:
predictions_v = df_val[['id', 'date', 'qty']]
predictions_v = pd.pivot(predictions_v, index = 'id', columns = 'date', values = 'qty').reset_index()

In [71]:
predictions_v['id'] = predictions['id'].apply(lambda x: x.replace('evaluation', 'validation'))

In [72]:
predictions_v.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [73]:
predictions_concat = pd.concat([predictions, predictions_v], axis=0)

In [75]:
predictions_concat.to_csv('C://Users/ismail/Documents/TP5/submission.csv', index=False)